Information gain is a splitting criterion that comes from information theory. It uses information entropy as the impurity function. 

O objetivo aqui é discorrer sobre os conceitos e implementações dos métodos de medição dos indicadores de impureza de uma amostra de dados, especificamente: 
- Entropy
- Gini index
- Caio Azevedo, May 22 2021

Principais referências:
- ref. https://www.featureranking.com/tutorials/machine-learning-tutorials/information-gain-computation/
- ref. https://machinelearningmastery.com/information-gain-and-mutual-information/
- ref. https://sites.math.washington.edu/~morrow/336_15/papers/lev.pdf
- ref. https://victorzhou.com/blog/information-gain/ (muito didático)
- ref. https://www.analyticssteps.com/blogs/what-gini-index-and-information-gain-decision-trees

In [13]:
# Primeira implementação usando apenas numpy e pandas
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

In [14]:
lst = ['apple']*3 + ['orange']*2 + ['banana']*2
fruits = pd.Series(lst)
print(fruits)

0     apple
1     apple
2     apple
3    orange
4    orange
5    banana
6    banana
dtype: object


In [15]:
# Computa a frequencia relativa para cada fruta na lista, que pode ser interpretada com a distribuição de probabilidade das frutas.
# Here is the relative frequency of each fruit in the basket, which can be considered as the probability distribution of the fruits.
probs = fruits.value_counts(normalize=True)
probs

apple     0.428571
banana    0.285714
orange    0.285714
dtype: float64

In [16]:
probs_by_hand = [3/7, 2/7, 2/7]
print(probs_by_hand)

[0.42857142857142855, 0.2857142857142857, 0.2857142857142857]


Given a probability distribution 𝑃 = (𝑝1, 𝑝2, . . , 𝑝𝑛), where 𝑝𝑖 is the probability that a point is in the subset 𝐷𝑖 of a dataset 𝐷, we define the entropy 𝐻:
Recall that Shannon's model defines entropy as
$$H(P) = -\sum_i^l{p_i} + log_2(p_i)$$
The idea with entropy is that the more heterogenous and impure a feature is, the higher the entropy. Conversely, the more homogenous and pure a feature is, the lower the entropy. The following calculation shows how impurity of this fruit basket can be computed using the entropy criterion.

In [17]:
entropy = -1 * np.sum(probs * np.log2(probs))
entropy

1.5566567074628228

The gini impurity index is defined as follows:
$$Gini(x) = 1-\sum_i^l{p_i}^2$$
The idea with Gini index is the same as in entropy in the sense that the more heterogenous and impure a feature is, the higher the Gini index. A nice property of the Gini index is that it is always between 0 and 1, and this may make it easier to compare Gini indices across different features. The impurity of our fruit basket using Gini index is calculated as below.

In [18]:
gini_index = 1 - np.sum(np.square(probs))
gini_index

0.653061224489796

In [19]:
#In comparison, let's compute impurity of another fruit basket with 7 different fruits with equal frequency.
lst2 = ['apple', 'orange', 'banana', 'mango', 'blueberry', 'watermelon', 'pear']
fruits2 = pd.Series(lst2)
print(fruits2)
probs2 = fruits2.value_counts(normalize=True)
probs2

0         apple
1        orange
2        banana
3         mango
4     blueberry
5    watermelon
6          pear
dtype: object


watermelon    0.142857
blueberry     0.142857
banana        0.142857
mango         0.142857
apple         0.142857
pear          0.142857
orange        0.142857
dtype: float64

In [20]:
entropy = -1 * np.sum(np.log2(probs2) * probs2)
entropy

2.807354922057604

In [21]:
gini_index = 1 - np.sum(np.square(probs2))
gini_index
#As expected, both entropy and Gini index of the second fruit basket is higher than those of the first fruit basket.

0.8571428571428572

In [22]:
#Agora vamos calcular os indíces de cada feature em um dataframe em relação a um target.
import pandas as pd
import io
import requests

# if you run into any SSL certification issues, 
# you may need to run the following command for a Mac OS installation.
# $/Applications/Python\ 3.6/Install\ Certificates.command

# how to read a csv file from a github account
df_url = 'https://raw.githubusercontent.com/cazevedo1977/datasets_2_ml/main/vegetation.csv'

url_content = requests.get(df_url).content
# print(url_content)
df = pd.read_csv(io.StringIO(url_content.decode('utf-8')))
df

,stream,slope,elevation,vegetation
0,False,steep,high,chapparal
1,True,moderate,low,riparian
2,True,steep,medium,riparian
3,False,steep,medium,chapparal
4,False,flat,high,conifer
5,True,steep,highest,conifer
6,True,steep,high,chapparal


In [23]:
def compute_impurity(feature, impurity_criterion):
    """
    This function calculates impurity of a feature.
    Supported impurity criteria: 'entropy', 'gini'
    input: feature (this needs to be a Pandas series)
    output: feature impurity
    """
    probs = feature.value_counts(normalize=True)
    
    if impurity_criterion == 'entropy':
        impurity = -1 * np.sum(probs * np.log2(probs))
    elif impurity_criterion == 'gini':
        impurity = 1 - np.sum(np.square(probs))
    else:
        raise ValueError('Unknown impurity criterion')
        
    return(round(impurity, 3))

# let's do two quick examples.
#print('impurity using entropy:', compute_impurity(fruits, 'entropy'))
#print('impurity using gini index:', compute_impurity(fruits, 'gini'))
# how to test for an incorrect compute_impurity_criterion value:
# print('impurity using gini index:', compute_impurity(df['stream'], 'foo'))

In [24]:
#Let's calculate entropy of the target feature "vegetation" using our new function.
target_entropy = compute_impurity(df['vegetation'], 'entropy')
target_entropy

1.557

Let's compute the information gain for splitting based on a descriptive feature to figure out the best feature to split on. For this task, we do the following:
- 1. Compute impurity of the target feature 'vegetation' (using either entropy or gini index).
- 2. Partition the dataset based on unique values of the descriptive feature.
- 3. Compute impurity (entropy) for each partition.
- 4. Compute the remaining impurity as the weighted sum of impurity of each partition.
- 5. Compute the information gain as the difference between the impurity of the target feature and the remaining impurity.

In case of impurity being entropy, it is possible to compute information gain ratio as well. If so, we do the following:
- 6. Compute the split_information defined by:
- 7. Divide information gain by the split_information

We will define another function to achieve this, called comp_feature_information_gain(). As an example, let's have a look at the levels of the "elevation" feature.

In [25]:
df['elevation'].value_counts()

high       3
medium     2
highest    1
low        1
Name: elevation, dtype: int64

In [26]:
##### Let's see how the partitions look like for this feature and what the corresponding calculations are using the entropy split criterion.
for level in df['elevation'].unique():
    print('level name:', level)
    df_feature_level = df[df['elevation'] == level]
    print('corresponding data partition:')
    print(df_feature_level)
    print('partition target feature impurity:', compute_impurity(df_feature_level['vegetation'], 'entropy'))
    print('partition weight:', str(len(df_feature_level)) + '/' + str(len(df)))
    print('====================')

level name: high
corresponding data partition:
   stream  slope elevation vegetation
0   False  steep      high  chapparal
4   False   flat      high    conifer
6    True  steep      high  chapparal
partition target feature impurity: 0.918
partition weight: 3/7
level name: low
corresponding data partition:
   stream     slope elevation vegetation
1    True  moderate       low   riparian
partition target feature impurity: -0.0
partition weight: 1/7
level name: medium
corresponding data partition:
   stream  slope elevation vegetation
2    True  steep    medium   riparian
3   False  steep    medium  chapparal
partition target feature impurity: 1.0
partition weight: 2/7
level name: highest
corresponding data partition:
   stream  slope elevation vegetation
5    True  steep   highest    conifer
partition target feature impurity: -0.0
partition weight: 1/7


The idea here is that, for each one of the 4 data partitions above,
- 1. We compute their impurity with respect to the target feature as a stand-alone dataset.
- 2. We weigh these impurities with the relative number of observations in each partition. The relative number of observations is calculated as the number of observations in the partition divided by the total number of observations in the entire dataset. For instance, the weight of the first partition is 3/7.
- 3. We add up these weighted impurities and call it the remaining impurity for this feature.

For instance, remaining impurity as measured by entropy for the elevation feature is 0.918 x (3/7) + 1.0 x (2/7) = 0.679. Information gain is then calculated as 1.557 - 0.679 = 0.878. Now we are ready to define our function. There is a bit of coding in here, but we can assure you that trying to figure out how things work in here will be rewarding to improve your Python programming skills.

In [27]:
from simple_colors import *
def comp_feature_information_gain(df, target, descriptive_feature, split_criterion):
    """
    This function calculates information gain for splitting on 
    a particular descriptive feature for a given dataset
    and a given impurity criteria.
    Supported split criterion: 'entropy', 'gini'
    """
    
    print('target feature:', target)
    print('descriptive_feature:', descriptive_feature)
    print('split criterion:', split_criterion)
            
    target_entropy = compute_impurity(df[target], split_criterion)
    print('target entropy:', target_entropy)

    # we define two lists below:
    # entropy_list to store the entropy of each partition
    # weight_list to store the relative number of observations in each partition
    entropy_list = list()
    weight_list = list()
        
    # loop over each level of the descriptive feature
    # to partition the dataset with respect to that level
    # and compute the entropy and the weight of the level's partition
    for level in df[descriptive_feature].unique():
        df_feature_level = df[df[descriptive_feature] == level]
        entropy_level = compute_impurity(df_feature_level[target], split_criterion)
        entropy_list.append(round(entropy_level, 3))
        weight_level = len(df_feature_level) / len(df)
        weight_list.append(round(weight_level, 3))
       
    print('impurity of partitions:', entropy_list)
    print('weights of partitions:', weight_list)
    
    feature_remaining_impurity = np.sum(np.array(entropy_list) * np.array(weight_list))
    print('remaining impurity:', feature_remaining_impurity)
    
    information_gain = target_entropy - feature_remaining_impurity
    print('information gain:', yellow("\033[1m" +  str(information_gain) + "\033[1m"))

    if split_criterion == 'entropy':
        split_info = compute_impurity(df[descriptive_feature], split_criterion)
        information_gain_ratio = information_gain / split_info
        print('split info:', split_info)
        print('information gain ratio:', green("\033[1m" +  str(information_gain_ratio) + "\033[1m"))
    
    print('====================')

    return(information_gain)

In [28]:
#Now that our function has been defined, we will call it for each descriptive feature in the dataset. First let's call it using the entropy split criteria.
split_criterion = 'entropy'
for feature in df.drop(columns='vegetation').columns:
    feature_info_gain = comp_feature_information_gain(df, 'vegetation', feature, split_criterion)

target feature: vegetation
descriptive_feature: stream
split criterion: entropy
target entropy: 1.557
impurity of partitions: [0.918, 1.5]
weights of partitions: [0.429, 0.571]
remaining impurity: 1.250322
information gain: 0.306678
split info: 0.985
information gain ratio: 0.3113482233502538
target feature: vegetation
descriptive_feature: slope
split criterion: entropy
target entropy: 1.557
impurity of partitions: [1.371, -0.0, -0.0]
weights of partitions: [0.714, 0.143, 0.143]
remaining impurity: 0.9788939999999999
information gain: 0.578106
split info: 1.149
information gain ratio: 0.5031383812010444
target feature: vegetation
descriptive_feature: elevation
split criterion: entropy
target entropy: 1.557
impurity of partitions: [0.918, -0.0, 1.0, -0.0]
weights of partitions: [0.429, 0.143, 0.286, 0.143]
remaining impurity: 0.6798219999999999
information gain: 0.877178
split info: 1.842
information gain ratio: 0.47620955483170463


In [29]:
#Now let's call it using the gini index split criteria.
#split_criteria = 'gini'
#for feature in df.drop(columns='vegetation').columns:
#    feature_info_gain = comp_feature_information_gain(df, 'vegetation', feature, split_criteria)

In [30]:
We observe that, with both the entropy and gini index split criteria, the highest information gain occurs with the "elevation" feature. This is the for the split at the root node of the corresponding decision tree. In subsequent splits, the above procedure is repeated with the subset of the entire dataset in the current branch until the termination condition is reached. 

SyntaxError: invalid syntax (<ipython-input-30-105066e0dde9>, line 1)

In [31]:
#Testando outro dataset didático
#ref. https://machinewithdata.com/2020/06/17/deriving-decision-tree-using-entropy-id3-approach/
#ref. https://machinewithdata.com/2018/07/10/how-to-calculate-gain-ratio/
df_url = 'https://raw.githubusercontent.com/cazevedo1977/datasets_2_ml/main/playornot2play.csv'

url_content = requests.get(df_url).content
# print(url_content)
df = pd.read_csv(io.StringIO(url_content.decode('utf-8')))
df

,outlook,temperature,humidity,wind,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [32]:
##### Let's see how the partitions look like for this feature and what the corresponding calculations are using the entropy split criterion.
for level in df['wind'].unique():
    print('level name:', level)
    df_feature_level = df[df['wind'] == level]
    print('corresponding data partition:')
    print(df_feature_level)
    print('partition target feature impurity:', compute_impurity(df_feature_level['play'], 'entropy'))
    print('partition weight:', str(len(df_feature_level)) + '/' + str(len(df)))
    print('====================')

level name: False
corresponding data partition:
     outlook temperature humidity   wind play
0      sunny         hot     high  False   no
2   overcast         hot     high  False  yes
3      rainy        mild     high  False  yes
4      rainy        cool   normal  False  yes
7      sunny        mild     high  False   no
8      sunny        cool   normal  False  yes
9      rainy        mild   normal  False  yes
12  overcast         hot   normal  False  yes
partition target feature impurity: 0.811
partition weight: 8/14
level name: True
corresponding data partition:
     outlook temperature humidity  wind play
1      sunny         hot     high  True   no
5      rainy        cool   normal  True   no
6   overcast        cool   normal  True  yes
10     sunny        mild   normal  True  yes
11  overcast        mild     high  True  yes
13     rainy        mild     high  True   no
partition target feature impurity: 1.0
partition weight: 6/14


In [33]:
#Now that our function has been defined, we will call it for each descriptive feature in the dataset. First let's call it using the entropy split criteria.
split_criterion = 'entropy'
for feature in df.drop(columns='play').columns:
    feature_info_gain = comp_feature_information_gain(df, 'play', feature, split_criterion)

target feature: play
descriptive_feature: outlook
split criterion: entropy
target entropy: 0.94
impurity of partitions: [0.971, -0.0, 0.971]
weights of partitions: [0.357, 0.286, 0.357]
remaining impurity: 0.693294
information gain: 0.24670599999999998
split info: 1.577
information gain ratio: 0.15644007609384908
target feature: play
descriptive_feature: temperature
split criterion: entropy
target entropy: 0.94
impurity of partitions: [1.0, 0.918, 0.811]
weights of partitions: [0.286, 0.429, 0.286]
remaining impurity: 0.9117679999999999
information gain: 0.028232000000000035
split info: 1.557
information gain ratio: 0.018132305716120768
target feature: play
descriptive_feature: humidity
split criterion: entropy
target entropy: 0.94
impurity of partitions: [0.985, 0.592]
weights of partitions: [0.5, 0.5]
remaining impurity: 0.7885
information gain: 0.15149999999999997
split info: 1.0
information gain ratio: 0.15149999999999997
target feature: play
descriptive_feature: wind
split criteri

In [34]:
#Agora fazendo uso da lib info_gain
from info_gain import info_gain

In [35]:
ig  = info_gain.info_gain(df['vegetation'], df['elevation'])
iv  = info_gain.intrinsic_value(df['vegetation'], df['elevation'])
igr = info_gain.info_gain_ratio(df['vegetation'], df['elevation']) # que é o mesmo que ig/iv
print(ig, iv, igr,ig/iv)

KeyError: 'vegetation'

In [36]:
ig  = info_gain.info_gain(df['vegetation'], df['slope'])
iv  = info_gain.intrinsic_value(df['vegetation'], df['slope'])
igr = info_gain.info_gain_ratio(df['vegetation'], df['slope'])
print(ig, iv, igr)

KeyError: 'vegetation'

In [37]:
ig  = info_gain.info_gain(df['vegetation'], df['stream'])
iv  = info_gain.intrinsic_value(df['vegetation'], df['stream'])
igr = info_gain.info_gain_ratio(df['vegetation'], df['stream'])
print(ig, iv, igr)

KeyError: 'vegetation'

In [ ]:
#Usando outro conjunto de dados para confrontar as duas implementações
produce = ['apple', 'apple', 'apple', 'strawberry', 'eggplant']
fruit   = [ True  ,  True  ,  True  ,  True       ,  False    ]
colour  = ['green', 'green', 'red'  , 'red'       , 'purple'  ]
ig  = info_gain.info_gain(fruit, colour)
iv  = info_gain.intrinsic_value(fruit, colour)
igr = info_gain.info_gain_ratio(fruit, colour)
print(ig, iv, igr)

In [ ]:
ig  = info_gain.info_gain(fruit, produce)
iv  = info_gain.intrinsic_value(fruit, produce)
igr = info_gain.info_gain_ratio(fruit, produce)
print(ig, iv, igr)

In [ ]:
df2 = pd.DataFrame(list(zip(fruit,colour,produce)),columns = ['fruit','colour','produce'])
df2

In [ ]:
split_criteria = 'entropy'
for feature in df2.drop(columns='fruit').columns:
    feature_info_gain = comp_feature_information_gain(df2, 'fruit', feature, split_criteria)